In [1]:
import os
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [2]:
import torch
from torch.utils.data import DataLoader

In [3]:
from utils.pytorch_datasets import Ben19Dataset
from utils.pytorch_models import ResNet18
from utils.pytorch_utils import start_cuda, get_classification_report, print_micro_macro
from utils.pytorch_utils import MetricTracker, init_results, update_results

## Parameters

In [4]:
from pathlib import Path

data_dir = Path("data/")

# path to LMDB file of all BEN19 patches
lmdb_path = data_dir / "BigEarth_Serbia_Summer_S2.lmdb/"
# csv paths to train & val file belonging to serbia
csv_train_path = data_dir / "train.csv"
csv_val_path = data_dir / "test.csv"

In [5]:
cuda_no = 1
batch_size = 128
num_workers = 0
epochs = 1

channels = 10
num_classes = 19
dataset_filter = "serbia"

## Initialize Model, Optimizer and Loss Function (Criterion)

## Initialize Train & Val Set and DataLoaders

In [6]:
training_set = Ben19Dataset(
    lmdb_path=lmdb_path, csv_path=csv_train_path, img_transform="default"
)
train_loader = DataLoader(
    training_set,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=True,
    pin_memory=True,
)

validation_set = Ben19Dataset(
    lmdb_path=lmdb_path, csv_path=csv_val_path, img_transform="default"
)
val_loader = DataLoader(
    validation_set,
    batch_size=batch_size,
    num_workers=num_workers,
    shuffle=False,
    pin_memory=True,
)

## Functions for Training

In [7]:
def train_epoch(model, train_loader, criterion, optimizer, epoch):
    loss_tracker = MetricTracker()
    model.train()

    for idx, batch in enumerate(tqdm(train_loader, desc="training")):
        data, labels, index = batch["data"], batch["label"], batch["index"]
        data = data
        labels = labels
        optimizer.zero_grad()

        logits = model(data)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        batch_size = data.size(0)
        loss_tracker.update(loss.item(), batch_size)

    print("Train loss: {:.6f}".format(loss_tracker.avg))
    return loss_tracker.avg

In [8]:
def val_epoch(model, val_loader, dataset_filter):
    model.eval()
    y_true = []
    predicted_probs = []

    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(val_loader, desc="test")):
            data = batch["data"]
            labels = batch["label"].numpy()

            logits = model(data)
            probs = torch.sigmoid(logits).cpu().numpy()

            predicted_probs += list(probs)

            y_true += list(labels)

    predicted_probs = np.asarray(predicted_probs)
    y_predicted = (predicted_probs >= 0.5).astype(np.float32)

    y_true = np.asarray(y_true)
    report = get_classification_report(
        y_true, y_predicted, predicted_probs, dataset_filter
    )
    return report

In [9]:
def train(
    model, train_loader, val_loader, criterion, optimizer, epochs, dataset_filter
):
    results = init_results(num_classes)

    for epoch in range(1, epochs + 1):
        print("Epoch {}/{}".format(epoch, epochs))
        print("-" * 10)

        train_epoch(model, train_loader, criterion, optimizer, epoch)
        report = val_epoch(model, val_loader, dataset_filter)

        results = update_results(results, report, num_classes)
        print_micro_macro(report)

    return results

### <span style="color:();">Run to silence warnings.</span>

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=UserWarning)  # Suppress UserWarnings
    warnings.filterwarnings("ignore", category=FutureWarning)  # Suppress FutureWarnings

### Select Model

Run one of the following cells to train the corresponding model

In [11]:
from models.poolformer import poolformer_s12

model = poolformer_s12(in_chans=10, num_classes=19)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
s12_results = train(
    model, train_loader, val_loader, criterion, optimizer, epochs, dataset_filter
)

Epoch 1/1
----------


training:   0%|          | 0/61 [00:00<?, ?it/s]

In [ ]:
from models.poolformer import poolformer_s24

model = poolformer_s24(in_chans=10, num_classes=19)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
s24_results = train(
    model, train_loader, val_loader, criterion, optimizer, epochs, dataset_filter
)

In [ ]:
from models.poolformer import poolformer_s36

model = poolformer_s36(in_chans=10, num_classes=19)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
s36_results = train(
    model, train_loader, val_loader, criterion, optimizer, epochs, dataset_filter
)

In [ ]:
from models.poolformer import poolformer_m36

model = poolformer_m36(in_chans=10, num_classes=19)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
m36_results = train(
    model, train_loader, val_loader, criterion, optimizer, epochs, dataset_filter
)

In [ ]:
from models.poolformer import poolformer_m48

model = poolformer_m48(in_chans=10, num_classes=19)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
m48_results = train(
    model, train_loader, val_loader, criterion, optimizer, epochs, dataset_filter
)

In [ ]:
model = ResNet18(num_cls=num_classes, channels=channels, pretrained=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0)
criterion = torch.nn.BCEWithLogitsLoss(reduction="mean")
resnet_results = train(
    model, train_loader, val_loader, criterion, optimizer, epochs, dataset_filter
)

/opt/homebrew/Caskroom/miniconda/base/envs/cv4rs/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/cv4rs/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
----------


training:   0%|          | 0/61 [00:00<?, ?it/s]

Train loss: 0.211027


test:   0%|          | 0/28 [00:00<?, ?it/s]

...

training:   0%|          | 0/61 [00:00<?, ?it/s]

Train loss: 0.028988


test:   0%|          | 0/28 [00:00<?, ?it/s]

micro     precision: 0.7527 | recall: 0.7353 | f1-score: 0.7439 | support: 9725 | mAP: 0.8275
macro     precision: 0.4938 | recall: 0.3858 | f1-score: 0.4125 | support: 9725 | mAP: 0.4626



/opt/homebrew/Caskroom/miniconda/base/envs/cv4rs/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/cv4rs/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/cv4rs/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:891: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/cv4rs/lib/python3.11/site-packages/sklearn/metri

In [ ]:
print(s12_results.keys())

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', 'micro avg', 'macro avg', 'ap_mic', 'ap_mac'])


In [ ]:
import pickle

with open("s12_results.pkl", "wb") as f:
    pickle.dump(s12_results, f)

with open("resnet_results.pkl", "wb") as f:
    pickle.dump(resnet_results, f)

In [ ]:
with open("s12_results.pkl", "rb") as f:
    loaded_dict = pickle.load(f)
    assert loaded_dict == s12_results